In [1]:
import pandas as pd 

In [28]:
import numpy as np 

In [29]:
from fuzzywuzzy import fuzz


C:\Users\rayen\Desktop\programming\big_data\torch_env\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import multiprocessing

In [4]:
profiles = pd.read_json('output.json',orient='records')  
jobs = pd.read_json('new_jobs.json',orient='records')   


In [60]:
def location_similiary(profile_loc, job_loc, threshold):
    
    similarity_score = fuzz.ratio(profile_loc.lower(), job_loc.lower())
    if similarity_score >= threshold : 
        return 1 
    return 0  
def level_similarity(profile_level,job_level) : 
    if job_level==profile_level : 
        return 1 
    return 0 
def experience_similiarity(job_exp,profile_exp) : 
    profile_years = int(profile_exp / 365) 
    if profile_years > job_exp[0] and profile_years < job_exp[1]  : 
        return 1 
    elif profile_years > job_exp[0] and profile_years > job_exp[1] : 
        return 1.2 
    else : 
        return 1 

def skills_similiarity(job_skills, profile_skills):
    # Convert lists to sets to remove duplicates and make order irrelevant
    job_skills_set = set(job_skills)
    profile_skills_set = set(profile_skills)
    
    # Compute intersection of skills
    intersecting_skills = job_skills_set.intersection(profile_skills_set)
    
    if not intersecting_skills:
        return 0.0  # If no intersecting skills, return 0 similarity
    
    # Create binary vectors representing skills
    job_skill_vector = np.array([1 if skill in job_skills_set else 0 for skill in job_skills_set])
    profile_skill_vector = np.array([1 if skill in intersecting_skills else 0 for skill in job_skills_set])
    
    # Reshape vectors for cosine similarity calculation
    job_skill_vector = job_skill_vector.reshape(1, -1)
    profile_skill_vector = profile_skill_vector.reshape(1, -1)
    
    # Compute cosine similarity between profile and job skills
    similarity = cosine_similarity(job_skill_vector, profile_skill_vector)[0][0]
    
    return similarity

def extract_similarity(job,profile,weights) : 
    skills_sem = skills_similiarity(job['skills'],profile['skills']) 
    location_sem = location_similiary(profile['location'],job['location'],0.9)
    exp_sim = experience_similiarity(job['experience'],profile['days_experience']) 
    level_sim = level_similarity(profile['max_level'],job['education']) 
    final_similarity = skills_sem * weights['skills'] + location_sem * weights['location'] + exp_sim * weights['experience'] + level_sim * weights['level'] 
    return final_similarity
def calculate_similarity_for_index(profile, job, idx):
    
    weights = {'skills' : 0.6,'location':0.1,'experience':0.2,'level':0.1}
    similarity_score = extract_similarity(job, profile,weights)
    return similarity_score, idx
def calculate_similarities_parallel(profiles, job):

    with multiprocessing.Pool() as pool:
        results = pool.starmap(calculate_similarity_for_index, [(profiles, job, idx) for idx in range(len(profiles))])
    return results
def calculate_similarities(profiles, job):
    
    similarities = []
    for idx, profile in enumerate(profiles):
            similarity_score = calculate_similarity_for_index(profile, job,idx)
            similarities.append((similarity_score, idx))
    return similarities

In [63]:
job = jobs.to_dict(orient='records')[2] 
profs = profiles.to_dict(orient='records')



In [15]:
profs = profiles.to_dict(orient='records')

In [64]:
results = calculate_similarities(profs,job)


In [65]:
results

[((0.33999999999999997, 0), 0),
 ((0.33999999999999997, 1), 1),
 ((0.33999999999999997, 2), 2),
 ((0.30000000000000004, 3), 3),
 ((0.33999999999999997, 4), 4),
 ((0.33999999999999997, 5), 5),
 ((0.33999999999999997, 6), 6),
 ((0.33999999999999997, 7), 7),
 ((0.33999999999999997, 8), 8),
 ((0.33999999999999997, 9), 9),
 ((0.33999999999999997, 10), 10),
 ((0.33999999999999997, 11), 11),
 ((0.33999999999999997, 12), 12),
 ((0.33999999999999997, 13), 13),
 ((0.33999999999999997, 14), 14),
 ((0.33999999999999997, 15), 15),
 ((0.33999999999999997, 16), 16),
 ((0.33999999999999997, 17), 17),
 ((0.33999999999999997, 18), 18),
 ((0.33999999999999997, 19), 19),
 ((0.33999999999999997, 20), 20),
 ((0.33999999999999997, 21), 21),
 ((0.33999999999999997, 22), 22),
 ((0.33999999999999997, 23), 23),
 ((0.30000000000000004, 24), 24),
 ((0.33999999999999997, 25), 25),
 ((0.33999999999999997, 26), 26),
 ((0.33999999999999997, 27), 27),
 ((0.33999999999999997, 28), 28),
 ((0.33999999999999997, 29), 29),


In [24]:
weights = {'skills' : 0.6,'location':0.1,'experience':0.2,'level':0.1}


In [33]:
extract_similarity(job, prof1,weights)

0.33999999999999997

In [38]:
type(prof1)

dict